# 1. Import Dependencies

In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

/home/pooya-fallah/python_envs/reinforce_practice/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Types of Spaces

In [20]:
Discrete(3).sample()

1

In [25]:
Box(0,1, shape=(3, 3)).sample()

array([[0.8776668 , 0.87673354, 0.5108979 ],
       [0.6482488 , 0.3620762 , 0.28817943],
       [0.3856471 , 0.39741725, 0.44026974]], dtype=float32)

In [32]:
Tuple((Discrete(3), Box(0,1, shape=(3,)))).sample()

(1, array([0.2533906 , 0.20176673, 0.56151587], dtype=float32))

In [34]:
Dict({"height":Discrete(2), "speed":Box(0,100,shape=(1,))}).sample()

OrderedDict([('height', 0), ('speed', array([21.722906], dtype=float32))])

In [37]:
MultiBinary(4).sample()

array([1, 0, 1, 0], dtype=int8)

In [41]:
MultiDiscrete([5,2,2]).sample()

array([0, 0, 0])

# 3. Building an Environment
- Build an agent that give us the best shower possible
- Randomly temperature
- 37 and 39 degree

In [68]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3, 3)
        # Set shower length
        self.shower_length = 60
    
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0
        # 2 -1 = 1 temperature
        self.state += action -1
        # Reduce shower length by 1 second
        self.shower_length -= 1
        
        # Calculate reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
            
        # Check if shower is done
        if self.shower_length <= 0:
            done = True
        else:
            done = False
            
        # Apply temperature noise
        # self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info
    
    def render(self, mod):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3, 3)]).astype('float32')
        # Reset shower time
        self.shower_length = 60
        return self.state

In [60]:
env = ShowerEnv()

In [52]:
env.observation_space.sample()

array([62.736515], dtype=float32)

In [53]:
env.reset()

array([39.])

In [54]:
from stable_baselines3.common.env_checker import check_env

In [61]:
check_env(env, warn=True)

# 4. Test Environment

In [62]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-40
Episode:2 Score:-12
Episode:3 Score:-48
Episode:4 Score:-16
Episode:5 Score:-8


# 5. Train Model

In [63]:
log_path = os.path.join('Training', 'Logs')

In [64]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [65]:
model.learn(total_timesteps=100000)

Logging to Training/Logs/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -32.2    |
| time/              |          |
|    fps             | 826      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -29.6       |
| time/                   |             |
|    fps                  | 701         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008610485 |
|    clip_fraction        | 0.0191      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.000601   |

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -25.4        |
| time/                   |              |
|    fps                  | 652          |
|    iterations           | 11           |
|    time_elapsed         | 34           |
|    total_timesteps      | 22528        |
| train/                  |              |
|    approx_kl            | 0.0071228007 |
|    clip_fraction        | 0.104        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.982       |
|    explained_variance   | -7.7e-05     |
|    learning_rate        | 0.0003       |
|    loss                 | 40.8         |
|    n_updates            | 100          |
|    policy_gradient_loss | -0.00826     |
|    value_loss           | 76.3         |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -10.3       |
| time/                   |             |
|    fps                  | 636         |
|    iterations           | 21          |
|    time_elapsed         | 67          |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.005051351 |
|    clip_fraction        | 0.0597      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.887      |
|    explained_variance   | -0.0109     |
|    learning_rate        | 0.0003      |
|    loss                 | 32.7        |
|    n_updates            | 200         |
|    policy_gradient_loss | -0.004      |
|    value_loss           | 67.1        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 15          |
| time/                   |             |
|    fps                  | 631         |
|    iterations           | 31          |
|    time_elapsed         | 100         |
|    total_timesteps      | 63488       |
| train/                  |             |
|    approx_kl            | 0.016701367 |
|    clip_fraction        | 0.219       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.876      |
|    explained_variance   | 0.000467    |
|    learning_rate        | 0.0003      |
|    loss                 | 28.4        |
|    n_updates            | 300         |
|    policy_gradient_loss | -0.0195     |
|    value_loss           | 63.6        |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60  

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60         |
|    ep_rew_mean          | 42.5       |
| time/                   |            |
|    fps                  | 639        |
|    iterations           | 41         |
|    time_elapsed         | 131        |
|    total_timesteps      | 83968      |
| train/                  |            |
|    approx_kl            | 0.01031692 |
|    clip_fraction        | 0.122      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.794     |
|    explained_variance   | 1.32e-05   |
|    learning_rate        | 0.0003     |
|    loss                 | 34.5       |
|    n_updates            | 400        |
|    policy_gradient_loss | 0.00516    |
|    value_loss           | 73.4       |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_m

# 6. Evaluate Model

In [73]:
evaluate_policy(model, env, n_eval_episodes=10)

(35.6, 47.80627573865171)